In [5]:
import rasterio
import pandas as pd
import numpy as np
import os

In [9]:
path = 'Y:\\PlanetWorldW\\Running\\FocusedAOIs\\Zambia\\ZM_AOI_1\\'

def statistics(path):
    AOI = path.rsplit('\\', 2)
    AOI = AOI[1]

#Load validation image

    with rasterio.open(path + 'classification_AOI.tif') as src:
        
        #image loading
        img_val = src.read(1)
        og_shape = img_val.shape
        profile = src.profile
        
        #array to list
        img_val = img_val.reshape(-1, 1)
        img_val = img_val.flatten()
        img_val_list = img_val.tolist()

        
        
        
    submissions = path + '\\Submissions\\'
    i = 0
    for filename in os.listdir(submissions):
        if filename.endswith(".tif"):
            
            name = filename[:-4]
            if (name == 'Waterloo') or (name == 'RSS'):
                print('1:',name)
                raster = os.path.join(submissions, filename)

                with rasterio.open(raster) as src:
                    img_sub = src.read(1)
                    img_sub = img_sub.reshape(-1, 1)
                    img_sub = img_sub.flatten()
                    img_sub_list = img_sub.tolist()

                # Validation
                A_k = img_val_list.count(1)
                # Submission
                A_e = img_sub_list.count(1)

                A_i = 0

                ## NEW RASTER ## 
                new_raster = []

            
                for pixel in range(len(img_val)):
                    if (img_val_list[pixel] < 1.5) & (img_sub_list[pixel] < 1.5 or img_sub_list[pixel] > 2.5):
                        #Negative
                        new_raster.append(0)
                    if (img_val_list[pixel] > 1.5) & (img_sub_list[pixel] > 1.5 and img_sub_list[pixel] < 2.5):
                        #Positive
                        new_raster.append(1)
                    if (img_val_list[pixel] > 1.5) & (img_sub_list[pixel] < 1.5 or img_sub_list[pixel] > 2.5):
                        #False Negative
                        new_raster.append(2)
                    if (img_val_list[pixel] < 1.5) & (img_sub_list[pixel] > 1.5 and img_sub_list[pixel] < 2.5):
                        #False Positive
                        new_raster.append(3)

                new_raster_arr = np.array(new_raster)
                new_raster_arr = new_raster_arr.reshape(og_shape)
                new_raster_arr = new_raster_arr.astype(np.uint8)
                print('test',path + AOI)        
                with rasterio.open(path +'\\Results\\'+name+'_result.tif', 'w', **profile) as dst:
                    dst.write(new_raster_arr, 1)


                #Statistics
                for pixel in range(len(img_val)):
                    if img_val_list[pixel] > 1.5 and img_sub_list[pixel] > 1.5 and img_sub_list[pixel] < 2.5:
                        A_i = A_i + 1


                A_max = 1.5 * ((A_i/A_e) + (A_i/A_k))
                A_cor = A_i/A_e
                A_com = A_i/A_k
                A_Qual = A_i / (A_e + A_k - A_i)

                data = {'Submission': [name],'A_max': [A_max], 'A_cor': [A_cor],'A_com': [A_com],'A_Qual': [A_Qual]}
                if i < 1.5:
                    df = pd.DataFrame(data=data)
                    df = df.round(3)
                    i = i+1
                else:
                    df2 = pd.DataFrame(data=data)
                    df = df.append(df2)
                    df = df.round(3)
    df = df.set_index('Submission')
    df.to_csv(path + AOI + "_statistics.csv")
    return df

In [10]:
a = statistics(path)

1: RSS
test Y:\PlanetWorldW\Running\FocusedAOIs\Zambia\ZM_AOI_1\ZM_AOI_1
1: Waterloo
test Y:\PlanetWorldW\Running\FocusedAOIs\Zambia\ZM_AOI_1\ZM_AOI_1


In [11]:
a

,A_max,A_cor,A_com,A_Qual
Submission,,,,
Waterloo,0.0,0.0,0.0,0.0
